# 07/04/2020

Source: https://docs.qiime2.org/2020.2/tutorials/moving-pictures/#alpha-and-beta-diversity-analysis

1. Alpha and beta diversity analyses
2. Ordination

Source: https://docs.qiime2.org/2020.2/tutorials/moving-pictures/#differential-abundance-testing-with-ancom

3. Differential abundance with ANCOM

In [1]:
cd /xdisk/tfaily/mig2020/extra/nathaliagg/sulfate_experiment/microbial_16S/qiime2

### Load module and activate conda environment

Disregard warnings.

In [2]:
module load anaconda/2020/2020.02

 /cm/local/apps/environment-modules/4.0.0//bin /cm/shared/uaapps/pbspro/19.2.4/sbin /cm/shared/uaapps/pbspro/19.2.4/bin
 /cm/shared/uaapps/pbspro/19.2.4/share/man


In [3]:
source activate qiime2-2020.2

(qiime2-2020.2) 

: 1

### 1. Alpha and beta diversity

Diversity analyses are available through the `q2-diversity plugin`, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations. 

First apply the `core-metrics-phylogenetic` method, which rarefies a `FeatureTable[Frequency]` to a user-specified depth, computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics. The metrics computed by default are:

- Alpha diversity
    - Shannon’s diversity index (a quantitative measure of community richness)
    - Observed OTUs (a qualitative measure of community richness)
    - Faith’s Phylogenetic Diversity (a qualitiative measure of community richness that incorporates phylogenetic relationships between the features)
    - Evenness (or Pielou’s Evenness; a measure of community evenness)


- Beta diversity
    - Jaccard distance (a qualitative measure of community dissimilarity)
    - Bray-Curtis distance (a quantitative measure of community dissimilarity)
    - unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)
    - weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

An important parameter that needs to be provided is `--p-sampling-depth`, which is the even sampling (i.e. alpha rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter. For example, if `--p-sampling-depth 500` is provided, this step will subsample the counts in each sample without replacement so that each sample in the resulting table has a total count of 500. If the total count for any sample(s) are smaller than this value, those samples will be dropped from the diversity analysis. **Choosing this value is tricky.** Make the choice by reviewing the information presented in the `table.qzv` file that was created previously. Choose a value that is as high as possible (so you retain more sequences per sample) while excluding as few samples as possible.

In [4]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny taxonomic_phylogenetic/rooted-tree.qza \
  --i-table denoise_dada2/table.qza \
  --p-sampling-depth 10000 \
  --m-metadata-file metadata.tsv \
  --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jacc

: 1

After computing diversity metrics, explore the microbial composition of the samples in the context of the sample metadata. This information is present in the sample metadata file.

First, test for associations between **categorical metadata** columns and alpha diversity data. Do that here for the Faith Phylogenetic Diversity (a measure of community richness) and evenness metrics:

In [5]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/observed_otus_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/observed_otus-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/shannon_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/shannon-group-significance.qzv
  
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/observed_otus-group-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/shannon-group-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/evenness-group-significance.qzv
(qiime2-2020.2) 

: 1

Second, test for associations between **continuous metadata** columns and alpha diversity. It's possible to do the code below changing the alpha metric in `--i-alpha-diversity` and visualizing the results.

In [ ]:
# qiime diversity alpha-correlation \
#     --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
#     --m-metadata-file metadata.tsv \
#     --p-method 'pearson' \
#     --o-visualization core-metrics-results/faith-pd-alpha-correlation.qzv

Next, analyze sample composition in the context of **categorical metadata** using PERMANOVA using the beta-group-significance command. The following commands will test whether distances between samples within a group, such as samples from the same site-name, are more similar to each other then they are to samples from the other groups. 

If the `--p-pairwise` parameter is called, it will also perform pairwise tests that will allow to determine which specific pairs of groups differ from one another, if any. This command can be slow to run, especially when passing `--p-pairwise`, since it is based on permutation tests. 

For this example, run beta-group-significance on a specific columns of metadata, rather than all metadata columns to which it is applicable. Here, apply this to unweighted UniFrac distances, using the `transect-name` and `vegetation`.

In [6]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Sulfate \
  --o-visualization core-metrics-results/unweighted-unifrac-SULFATE-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Temperature \
  --o-visualization core-metrics-results/unweighted-unifrac-TEMPERATURE-significance.qzv \
  --p-pairwise
  
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Time \
  --o-visualization core-metrics-results/unweighted-unifrac-TIME-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-SULFATE-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/unweighted-unifrac-TEMPERATURE-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/unweighted-unifrac-TIME-significance.qzv
(qiime2-2020.2) 

: 1

In [7]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Sulfate \
  --o-visualization core-metrics-results/weighted-unifrac-SULFATE-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Temperature \
  --o-visualization core-metrics-results/weighted-unifrac-TEMPERATURE-significance.qzv \
  --p-pairwise
  
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Time \
  --o-visualization core-metrics-results/weighted-unifrac-TIME-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/weighted-unifrac-SULFATE-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/weighted-unifrac-TEMPERATURE-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/weighted-unifrac-TIME-significance.qzv
(qiime2-2020.2) 

: 1

In [8]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Sulfate \
  --o-visualization core-metrics-results/bray_curtis-SULFATE-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Temperature \
  --o-visualization core-metrics-results/bray_curtis-TEMPERATURE-significance.qzv \
  --p-pairwise
  
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Time \
  --o-visualization core-metrics-results/bray_curtis-TIME-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/bray_curtis-SULFATE-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/bray_curtis-TEMPERATURE-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/bray_curtis-TIME-significance.qzv
(qiime2-2020.2) 

: 1

In [9]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/jaccard_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Sulfate \
  --o-visualization core-metrics-results/jaccard-SULFATE-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/jaccard_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Temperature \
  --o-visualization core-metrics-results/jaccard-TEMPERATURE-significance.qzv \
  --p-pairwise
  
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/jaccard_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Time \
  --o-visualization core-metrics-results/jaccard-TIME-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/jaccard-SULFATE-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/jaccard-TEMPERATURE-significance.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: core-metrics-results/jaccard-TIME-significance.qzv
(qiime2-2020.2) 

: 1

For the **continuous metadata** correlation tests, you can use the `qiime metadata distance-matrix` in combination with `qiime diversity mantel` and `qiime diversity bioenv` commands.

### 2. Ordination

Ordination is a popular approach for exploring microbial community composition in the context of sample metadata. Use the Emperor tool to explore principal coordinates (PCoA) plots in the context of sample metadata. 

Passing the optional parameter `--p-custom-axes` is very useful for exploring time series data. Emperor plots for unweighted UniFrac and Bray-Curtis are generating here, and the resulting plots will contain axes for principal coordinate 1, principal coordinate 2, and a third axis based on what passed in `--p-custom-axes`.

In [ ]:
# qiime emperor plot \
#   --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \
#   --m-metadata-file metadata.tsv \
#   --o-visualization core-metrics-results/unweighted-unifrac-emperor.qzv

# qiime emperor plot \
#   --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
#   --m-metadata-file metadata.tsv \
#   --p-custom-axes Temperature \
#   --o-visualization core-metrics-results/bray-curtis-emperor-TEMPERATURE.qzv
  
# qiime emperor plot \
#   --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
#   --m-metadata-file metadata.tsv \
#   --p-custom-axes Time \
#   --o-visualization core-metrics-results/bray-curtis-emperor-TIME.qzv

# 3. Differential abundance with ANCOM

ANCOM can be applied to identify features that are differentially abundant (i.e. present in different abundances) across sample groups. As with any bioinformatics method, you should be aware of the assumptions and limitations of ANCOM before using it (check the paper: https://pubmed.ncbi.nlm.nih.gov/26028277/).

ANCOM is implemented in the `q2-composition plugin`. ANCOM assumes that few (less than about 25%) of the features are changing between groups. If you expect that more features are changing between your groups, you should not use ANCOM as it will be more error-prone (an increase in both Type I and Type II errors is possible). 

We’ll then apply ANCOM to determine which, if any, sequence variants and genera are differentially abundant across samples of our two subjects.

We’ll start by creating a feature table that contains only the metadata of interest samples:

In [12]:
mkdir ancom-results

(qiime2-2020.2) 

: 1

In [13]:
# qiime feature-table filter-samples \
#   --i-table denoise_dada2/table.qza \
#   --m-metadata-file metadata.tsv \
#   --p-where "[Time]='T1'" \
#   --o-filtered-table ancom-results/time-t1-table.qza

Saved FeatureTable[Frequency] to: ancom-results/time-t1-table.qza
(qiime2-2020.2) 

: 1

ANCOM operates on a `FeatureTable[Composition]` QIIME2 artifact, which is based on frequencies of features on a per-sample basis, but cannot tolerate frequencies of zero. To build the composition artifact, a `FeatureTable[Frequency]` artifact must be provided to add-pseudocount (an imputation method), which will produce the `FeatureTable[Composition]` artifact:

In [14]:
# qiime composition add-pseudocount \
#   --i-table ancom-results/time-t1-table.qza \
#   --o-composition-table ancom-results/coda-time-t1-yes-table.qza

Saved FeatureTable[Composition] to: ancom-results/coda-time-t1-yes-table.qza
(qiime2-2020.2) 

: 1

Run ANCOM on the subject column to determine what features differ in abundance across the Time=T1 samples of the Sulfate:

In [16]:
# qiime composition ancom \
#   --i-table ancom-results/coda-time-t1-yes-table.qza \
#   --m-metadata-file metadata.tsv \
#   --m-metadata-column Sulfate \
#   --o-visualization ancom-results/time-t1-sulfate.qzv


Aborted!
(qiime2-2020.2) 

Done!

In [17]:
source deactivate qiime2-2020.2